In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ======================
# Dataset
# ======================
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5071, 0.4867, 0.4408),
        std=(0.2675, 0.2565, 0.2761)
    )
])

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5071, 0.4867, 0.4408),
        std=(0.2675, 0.2565, 0.2761)
    )
])

trainset = torchvision.datasets.CIFAR100(
    "./data", train=True, download=True, transform=train_transform
)
testset = torchvision.datasets.CIFAR100(
    "./data", train=False, download=True, transform=test_transform
)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# ======================
# Model (FREEZE layer2)
# ======================
model = resnet18(pretrained=True)

# Replace final layer for CIFAR-100
model.fc = nn.Linear(512, 100)

# 🔒 Freeze layer2
for param in model.layer2.parameters():
    param.requires_grad = False

model = model.to(device)

criterion = nn.CrossEntropyLoss()

# Train only unfrozen parameters
optimizer = torch.optim.AdamW(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=3e-4,
    weight_decay=1e-4
)

# ======================
# Accuracy function
# ======================
def accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            preds = logits.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)
    return 100 * correct / total

# ======================
# Training Loop
# ======================
EPOCHS = 10

print("\nFINETUNING WITH layer2 FROZEN")

for epoch in range(EPOCHS):
    model.train()
    loss_sum = 0.0

    for x, y in trainloader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    train_acc = accuracy(model, trainloader)
    test_acc = accuracy(model, testloader)

    print(
        f"Epoch {epoch+1}/{EPOCHS} | "
        f"Loss {loss_sum/len(trainloader):.4f} | "
        f"Train Acc {train_acc:.2f}% | "
        f"Test Acc {test_acc:.2f}%"
    )


Using device: cuda


100%|██████████| 169M/169M [00:04<00:00, 35.1MB/s] 
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 211MB/s]



FINETUNING WITH layer2 FROZEN
Epoch 1/10 | Loss 1.5514 | Train Acc 75.63% | Test Acc 70.71%
Epoch 2/10 | Loss 0.8160 | Train Acc 80.71% | Test Acc 73.15%
Epoch 3/10 | Loss 0.6207 | Train Acc 84.98% | Test Acc 75.17%
Epoch 4/10 | Loss 0.4846 | Train Acc 88.50% | Test Acc 76.18%
Epoch 5/10 | Loss 0.3866 | Train Acc 90.51% | Test Acc 76.75%
Epoch 6/10 | Loss 0.3209 | Train Acc 90.81% | Test Acc 75.80%
Epoch 7/10 | Loss 0.2637 | Train Acc 92.67% | Test Acc 76.16%
Epoch 8/10 | Loss 0.2210 | Train Acc 93.66% | Test Acc 76.51%
Epoch 9/10 | Loss 0.1994 | Train Acc 94.82% | Test Acc 77.29%
Epoch 10/10 | Loss 0.1699 | Train Acc 95.71% | Test Acc 77.72%
